In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Book2.csv to Book2.csv


In [ ]:
import pandas as pd

df = pd.read_csv("/content/Book2.csv", nrows=50,sep=",", encoding='cp1252')
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000)
vectors = vectorizer.fit_transform(df.Speech)
words_df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names_out())
X = words_df
y = df.Intent
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
nm = NearMiss()
X,y=ros.fit_resample(X,y)
X,y=nm.fit_resample(X,y)
bayes = MultinomialNB()
bayes.fit(X, y)
pd.set_option("display.max_colwidth", 200)


In [ ]:
uw=input("Enter Message:")

unknown=pd.DataFrame({'content': [uw]})
unknown_vectors = vectorizer.transform(unknown.content)
unknown_words_df = pd.DataFrame(unknown_vectors.toarray(), columns=vectorizer.get_feature_names_out())
unknown_words_df.head()
unknown['pred_logreg'] = bayes.predict(unknown_words_df)
unknown['pred_logreg_proba'] = bayes.predict_proba(unknown_words_df)[:,1]
unknown

Enter Message:search wikipedia for email


,content,pred_logreg,pred_logreg_proba
0,search wikipedia for email,Wikipedia,0.214018


In [ ]:
import pickle
filename = 'intent.sav'
pickle.dump(bayes, open(filename, 'wb'))



In [ ]:
import pickle
filename = 'intent.sav'
bayes_save = pickle.load(open(filename, 'rb'))
uw=input("Enter Message:")

unknown=pd.DataFrame({'content': [uw]})
unknown_vectors = vectorizer.transform(unknown.content)
unknown_words_df = pd.DataFrame(unknown_vectors.toarray(), columns=vectorizer.get_feature_names())
unknown_words_df.head()
unknown['pred_logreg'] = bayes_save.predict(unknown_words_df)
unknown['pred_logreg_proba'] = bayes_save.predict_proba(unknown_words_df)[:,1]
unknown

Enter Message:search


,content,pred_logreg,pred_logreg_proba
0,search,Wikipedia,0.170115


In [ ]:
import warnings
warnings.filterwarnings("ignore")
t=0
df1 = pd.read_csv("/content/IntentTestData.csv", nrows=50,sep=",", encoding='cp1252')
for i in range(32):
  D=df1.sample()
  uw1=D.Speech
  i1=D.Intent
  i2=i1.values
  uw2=uw1.values
  unknown=pd.DataFrame({'content': uw2})
  unknown_vectors = vectorizer.transform(unknown.content)
  unknown_words_df = pd.DataFrame(unknown_vectors.toarray(), columns=vectorizer.get_feature_names())
  unknown_words_df.head()
  unknown['pred_logreg'] = bayes_save.predict(unknown_words_df)
  if i2==unknown['pred_logreg'].values:
    t+=1
print(f'The Accuracy is:{100*t/32}%')




The Accuracy is:100.0%


In [ ]:
pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=4e71a2ffaaa1c6a40e7ca7233b0b87347dfbb1809114b2d94245df6f9f088d56
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [ ]:
import warnings
warnings.filterwarnings("ignore")
uw=input("Enter Message:")
unknown=pd.DataFrame({'content': [uw]})
unknown_vectors = vectorizer.transform(unknown.content)
unknown_words_df = pd.DataFrame(unknown_vectors.toarray(), columns=vectorizer.get_feature_names())
unknown_words_df.head()
unknown['pred_logreg'] = bayes_save.predict(unknown_words_df)
if "Wikipedia" == unknown['pred_logreg'].values:
   print('Y')
   uw=uw.lower()

   uw= uw.replace('wikipedia for','')
   uw= uw.replace('according to wikipedia','')
   uw=uw.replace('wiki','')
   uw= uw.replace('search','')
   uw= uw.replace('for','')
   uw=uw.replace(' ','_')

   print(uw)
   import wikipedia
   try:
    print("WWWW")
    s=wikipedia.summary(uw,sentences=3)
    print(s)
   except wikipedia.DisambiguationError as e:
    print("There are more than one such topics")
    print(e.options)
    uw = ''.join(uw.split())
    print(f"Please visit this link for articles on all topics related to {uw}:")
    print(f"https://en.wikipedia.org/wiki/{uw}")


Enter Message:search wiki for photography
Y
___photography
WWWW
Photography is the art, application, and practice of creating durable images by recording light, either electronically by means of an image sensor, or chemically by means of a light-sensitive material such as photographic film. It is employed in many fields of science, manufacturing (e.g., photolithography), and business, as well as its more direct uses for art, film and video production, recreational purposes, hobby, and mass communication.Typically, a lens is used to focus the light reflected or emitted from objects into a real image on the light-sensitive surface inside a camera during a timed exposure. With an electronic image sensor, this produces an electrical charge at each pixel, which is electronically processed and stored in a digital image file for subsequent display or processing.
